In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
from scipy import stats
from figure_making import data_loader

In [ ]:
animal_ids = ["SZ036", "SZ037", "SZ038", "SZ039", "SZ042", "SZ043"]
# animal_ids=["SZ036"]
master_df1 = data_loader.load_dataframes_for_animal_summary(animal_ids, 'nonreward_DA',
                                                            day_0='2023-11-30',
                                                            melt=1,
                                                            hemisphere_qc=1,
                                                            file_format='parquet')

animal_ids = ["RK007", "RK008", "RK009", "RK010"]
master_df2 = data_loader.load_dataframes_for_animal_summary(animal_ids, 'nonreward_DA',
                                                            day_0='2025-06-17',
                                                            melt=1,
                                                            hemisphere_qc=1,
                                                            file_format='parquet')
master_NR_DA_df = pd.concat([master_df1, master_df2], ignore_index=True)

In [ ]:
def bin_nonreward_DA(nonreward_DA_df, bin_size=0.1):
    max_time = nonreward_DA_df['time_in_port'].max()
    bins = np.arange(0, max_time + bin_size, bin_size)
    nonreward_DA_df['time_bin'] = pd.cut(nonreward_DA_df['time_in_port'], bins=bins, include_lowest=True)

    def sem(series):
        return series.std() / np.sqrt(len(series))

    binned_results = nonreward_DA_df.groupby(['animal', 'session', 'hemisphere', 'time_bin', 'phase'], observed=True)['DA'].agg(
        ['mean', ('sem', sem)]).reset_index()
    binned_results['bin_center'] = binned_results['time_bin'].apply(lambda x: x.mid.round(2))

    return binned_results

In [ ]:
grouped_df = bin_nonreward_DA(master_NR_DA_df, bin_size=0.5)

In [ ]:
def plot_phase_differences(binned_df, save=0):
    # --- 1. PRE-PROCESSING ---
    binned_df['bin_center'] = binned_df['bin_center'].astype(float)
    df = binned_df.copy()
    # df = binned_df[binned_df['bin_center'] <= 1.5].copy()
    # Pivot to get phases into columns
    df_pivot = df.pivot_table(
        index=['animal', 'session', 'hemisphere', 'bin_center'],
        columns='phase',
        values='mean'
    ).reset_index()
    # Calculate the difference (Phase 0.8 - Phase 0.4)
    df_pivot['diff'] = df_pivot['0.8'] - df_pivot['0.4']
    df_pivot = df_pivot.dropna(subset=['diff'])

    # --- 2. PLOTTING SETUP ---
    groups = df_pivot[['animal', 'hemisphere']].drop_duplicates()

    # Create a figure
    num_plots = len(groups)
    if num_plots == 0:
        print("No data found matching criteria.")
        return

    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(30, 20), squeeze=False)
    mpl.rcParams['figure.dpi'] = 300
    axes = axes.flatten() # Make axes easy to iterate over

    # --- 3. PLOTTING LOOP ---

    for i, (idx, row) in enumerate(groups.iterrows()):
        animal = row['animal']
        hemi = row['hemisphere']
        ax = axes[i]
        subset = df_pivot[(df_pivot['animal'] == animal) & (df_pivot['hemisphere'] == hemi)]
        sessions = sorted(subset['session'].unique())

        # --- GENERATE COLOR MAP ---
        colors = cm.Greys(np.linspace(0.3, 0.9, len(sessions)))

        # Plot individual sessions
        for j, session in enumerate(sessions):
            sess_data = subset[subset['session'] == session]
            ax.plot(
                sess_data['bin_center'],
                sess_data['diff'],
                color=colors[j],
                alpha=0.5,
                linewidth=1,
                label='_nolegend_'
            )

        # --- CALCULATE AND PLOT GRAND AVERAGE (RED) ---
        stats = subset.groupby('bin_center')['diff'].agg(['mean', 'sem']).reset_index()

        ax.plot(stats['bin_center'], stats['mean'], color='red', linewidth=2, label='Mean Diff')
        ax.fill_between(
            stats['bin_center'],
            stats['mean'] - stats['sem'],
            stats['mean'] + stats['sem'],
            color='red',
            alpha=0.2
        )

        # Formatting
        ax.set_title(f"{animal} - {hemi}")
        ax.set_xlabel("Time in Port (s)")
        ax.set_ylabel("DA Diff (High - Low)")
        ax.axhline(0, color='black', linestyle='--', linewidth=0.8) # Zero reference line

    plt.tight_layout()
    if save:
        plt.savefig('NR_rising_tide_1stmoment.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
plot_phase_differences(grouped_df, save=0)